# LOADING NECESSARY LIBRARIES

In [135]:
!pip install scikit-learn
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# DATA COLLECTION

In [136]:
# Loading the symptoms dataset
df = pd.read_csv("symtoms_df.csv")
medicine=pd.read_csv("medications1.csv.csv")
workout=pd.read_csv("workout_df.csv")
precaution = pd.read_csv("precautions_df.csv")
diet=pd.read_csv("diets.csv")


In [137]:
df.head(10)

,Unnamed: 0,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4
0,0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches
1,1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN
2,2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN
3,3,Fungal infection,itching,skin_rash,dischromic _patches,NaN
4,4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN
5,5,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN
6,6,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN
7,7,Fungal infection,itching,skin_rash,dischromic _patches,NaN
8,8,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN
9,9,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches


In [138]:
#loading medicine data set
medicine=pd.read_csv("medications1.csv.csv")

In [139]:
medicine.head(3)

,Disease,Medication
0,Fungal infection,"['Antifungal Cream', 'Fluconazole', 'Terbinafi..."
1,Allergy,"['Antihistamines', 'Decongestants', 'Epinephri..."
2,GERD,"['Proton Pump Inhibitors (PPIs)', 'H2 Blockers..."


In [140]:
precaution = pd.read_csv("precautions_df.csv")

In [141]:
precaution.head(3)

,Unnamed: 0,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,2,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching


In [142]:
workout=pd.read_csv("workout_df.csv")

In [143]:
workout.head(2)

,Unnamed: 0.1,Unnamed: 0,disease,workout
0,0,0,Fungal infection,Avoid sugary foods
1,1,1,Fungal infection,Consume probiotics


In [144]:
df.drop(workout['Unnamed: 0.1'],inplace=True)


In [145]:
grouped_workout = workout.groupby('disease')['workout'].agg(list).reset_index()

In [146]:
grouped_workout.head(7)

,disease,workout
0,(vertigo) Paroymsal Positional Vertigo,"[Avoid trigger foods (caffeine, alcohol), Limi..."
1,AIDS,"[Follow a balanced and nutritious diet, Includ..."
2,Acne,"[Consume a balanced diet, Limit dairy and high..."
3,Alcoholic hepatitis,"[Avoid alcohol consumption, Follow a balanced ..."
4,Allergy,"[Avoid allergenic foods, Consume anti-inflamma..."
5,Arthritis,"[Consume anti-inflammatory foods, Include omeg..."
6,Bronchial Asthma,"[Include anti-inflammatory foods, Consume omeg..."


In [147]:
diet=pd.read_csv("diets.csv")


In [148]:
diet.head(5)


,Disease,Diet
0,Fungal infection,"['Antifungal Diet', 'Probiotics', 'Garlic', 'C..."
1,Allergy,"['Elimination Diet', 'Omega-3-rich foods', 'Vi..."
2,GERD,"['Low-Acid Diet', 'Fiber-rich foods', 'Ginger'..."
3,Chronic cholestasis,"['Low-Fat Diet', 'High-Fiber Diet', 'Lean prot..."
4,Drug Reaction,"['Antihistamine Diet', 'Omega-3-rich foods', '..."


# DATA ANALYSIS 

In [149]:
df.describe()

,Unnamed: 0
count,4510.000000
mean,2664.500000
std,1302.069187
min,410.000000
25%,1537.250000
50%,2664.500000
75%,3791.750000
max,4919.000000


In [150]:
df.shape

(4510, 6)

In [151]:
df.isnull().sum()

Unnamed: 0      0
Disease         0
Symptom_1       0
Symptom_2       0
Symptom_3       0
Symptom_4     290
dtype: int64

In [152]:
df.nunique()

Unnamed: 0    4510
Disease         41
Symptom_1       34
Symptom_2       48
Symptom_3       54
Symptom_4       50
dtype: int64

In [153]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4510 entries, 410 to 4919
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Disease    4510 non-null   object
 1   Symptom_1  4510 non-null   object
 2   Symptom_2  4510 non-null   object
 3   Symptom_3  4510 non-null   object
 4   Symptom_4  4220 non-null   object
dtypes: object(5)
memory usage: 176.3+ KB


# DATA PRE-PROCESSING

In [155]:
# One-Hot Encoding of symptoms
encoder = OneHotEncoder(sparse_output=False)  # Set sparse=False to return a dense array
encoded = encoder.fit_transform(df[['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4']])

# Convert encoded data to DataFrame
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out())
encoded_df.columns = [col.replace('Symptom_4_', '') for col in encoded_df.columns]
encoded_df.columns = [col.replace('Symptom_1_', '') for col in encoded_df.columns]
encoded_df.columns = [col.replace('Symptom_3_', '') for col in encoded_df.columns]
encoded_df.columns = [col.replace('Symptom_2_', '') for col in encoded_df.columns]


# Display the encoded columns
encoded_df.columns

Index([' acidity', ' back_pain', ' bladder_discomfort', ' breathlessness',
       ' burning_micturition', ' chest_pain', ' chills', ' constipation',
       ' continuous_sneezing', ' cough',
       ...
       ' swelling_joints', ' swelling_of_stomach', ' swollen_legs',
       ' vomiting', ' watering_from_eyes', ' weight_loss',
       ' yellow_crust_ooze', ' yellowing_of_eyes', ' yellowish_skin', 'nan'],
      dtype='object', length=187)

In [156]:
#pip install pandas openpyxl

In [157]:

# Specify the path and filename for the Excel file
#excel_file_path = 'output.xlsx'

# Save the DataFrame to an Excel file
#encoded_df.to_excel(excel_file_path, index=False, engine='openpyxl')

In [158]:
#encoded_df=pd.read_excel("output.xlsx")

In [159]:
encoded_df

,acidity,back_pain,bladder_discomfort,breathlessness,burning_micturition,chest_pain,chills,constipation,continuous_sneezing,cough,...,swelling_joints,swelling_of_stomach,swollen_legs,vomiting,watering_from_eyes,weight_loss,yellow_crust_ooze,yellowing_of_eyes,yellowish_skin,nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4507,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [160]:
encoded_df.drop(['nan'],axis='columns',inplace=True)

In [161]:
#encoded_df.to_csv('output.csv', index=False)

In [162]:
# Extract the target variable
y = df['Disease']

# Label encode the diseases
le = LabelEncoder()
disease = le.fit_transform(df['Disease'])

# Display encoded diseases
disease


array([15, 15, 15, ..., 38, 35, 27])

# TRAIN-TEST SPLIT

In [163]:
# Splitting the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(encoded_df, disease, test_size=0.3, random_state=1)


In [164]:
#from sklearn.ensemble import GradientBoostingClassifier

In [165]:


# Ensure shapes match before splitting
#print(f"Encoded features shape: {encoded_df.shape}")


# Train-test split (ensure samples count match)
#X_train, X_test, y_train, y_test = train_test_split(encoded_df, disease, test_size=0.3, random_state=1)

# Initialize and train the Gradient Boosting model
#model = GradientBoostingClassifier(n_estimators=150, learning_rate=0.1, max_depth=3, random_state=42)
#model.fit(X_train, y_train)

# Make predictions
#y_pred = model.predict(X_train)
#print(y_pred)
#y_test_pred=model.predict(X_test)


In [166]:
#model.score(X_test,y_test)

In [167]:
#print(confusion_matrix(y_test, y_test_pred))

In [168]:
#print(classification_report(y_test,y_test_pred))

In [169]:
# RandomForest Classifier model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predicting on test data
y_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Accuracy scores
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
train_accuracy = accuracy_score(y_train, y_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Display train and test accuracy
train_accuracy, test_accuracy


(0.9952486537852392, 0.9963045084996305)

 # CONFUSION MATRIX AND CLASSIFICATION REPORT

In [170]:
# Confusion matrix and classification report
print(confusion_matrix(y_test, y_test_pred))


[[35  0  0 ...  0  0  0]
 [ 0 27  0 ...  0  0  0]
 [ 0  0 33 ...  0  0  0]
 ...
 [ 0  0  0 ... 29  0  0]
 [ 0  0  0 ...  0 37  0]
 [ 0  0  0 ...  0  0 34]]


In [171]:
print(classification_report(y_test, y_test_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00        27
           2       1.00      1.00      1.00        33
           3       1.00      1.00      1.00        28
           4       1.00      1.00      1.00        33
           5       1.00      1.00      1.00        37
           6       0.97      1.00      0.98        30
           7       1.00      1.00      1.00        29
           8       1.00      1.00      1.00        39
           9       1.00      1.00      1.00        32
          10       1.00      0.97      0.99        39
          11       1.00      1.00      1.00        35
          12       1.00      1.00      1.00        32
          13       1.00      1.00      1.00        33
          14       1.00      1.00      1.00        31
          15       1.00      1.00      1.00        39
          16       1.00      1.00      1.00        28
          17       1.00    

In [172]:
#import matplotlib as m
#import seaborn as sns

In [173]:
#sns.heatmap(confusion_matrix(y_test, y_test_pred),annot=True)

In [174]:
confusion_matrix(y_test,y_test_pred)[0]

array([35,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0], dtype=int64)

In [175]:
# Function to get the key for a given value
def get_disease_by_value(disease_map, value):
    for key, val in disease_map.items():
        if val == value:
            return key
    return "Disease not found"

# Example usage with predictions
diseasemap = {
    '(vertigo) Paroymsal  Positional Vertigo': 0, 'AIDS': 1, 'Acne': 2, 
    'Alcoholic hepatitis': 3, 'Allergy': 4, 'Arthritis': 5, 'Bronchial Asthma': 6,
    'Cervical spondylosis': 7, 'Chicken pox': 8, 'Chronic cholestasis': 9, 
    'Common Cold': 10, 'Dengue': 11, 'Diabetes': 12, 'Dimorphic hemmorhoids(piles)': 13, 
    'Drug Reaction': 14, 'Fungal infection': 15, 'GERD': 16, 'Gastroenteritis': 17, 
    'Heart attack': 18, 'Hepatitis B': 19, 'Hepatitis C': 20, 'Hepatitis D': 21, 
    'Hepatitis E': 22, 'Hypertension': 23, 'Hyperthyroidism': 24, 'Hypoglycemia': 25, 
    'Hypothyroidism': 26, 'Impetigo': 28, 'Jaundice': 29, 'Malaria': 30, 'Migraine': 31, 
    'Osteoarthristis': 32, 'Paralysis (brain hemorrhage)': 33, 'Peptic ulcer disease': 34, 
    'Pneumonia': 35, 'Psoriasis': 36, 'Tuberculosis': 37, 'Typhoid': 38, 
    'Urinary tract infection': 39, 'Varicose veins': 40, 'hepatitis A': 41
}

# Assuming model.predict() returns an array of predictions
predicted_values = model.predict(encoded_df.iloc[1:200])

# Iterate through predicted values and get corresponding disease names
predicted_diseases = [get_disease_by_value(diseasemap, value) for value in predicted_values]

# Output the predicted diseases
for disease in predicted_diseases:
    print(disease)

Fungal infection
Fungal infection
Fungal infection
Fungal infection
Fungal infection
Fungal infection
Fungal infection
Fungal infection
Fungal infection
Allergy
Allergy
Allergy
Allergy
Allergy
Allergy
Allergy
Allergy
Allergy
Allergy
GERD
GERD
GERD
GERD
GERD
GERD
GERD
GERD
GERD
GERD
Chronic cholestasis
Chronic cholestasis
Chronic cholestasis
Chronic cholestasis
Chronic cholestasis
Chronic cholestasis
Chronic cholestasis
Chronic cholestasis
Chronic cholestasis
Chronic cholestasis
Drug Reaction
Drug Reaction
Drug Reaction
Drug Reaction
Drug Reaction
Drug Reaction
Drug Reaction
Drug Reaction
Drug Reaction
Drug Reaction
Paralysis (brain hemorrhage)
Paralysis (brain hemorrhage)
Paralysis (brain hemorrhage)
Paralysis (brain hemorrhage)
Paralysis (brain hemorrhage)
Paralysis (brain hemorrhage)
Paralysis (brain hemorrhage)
Paralysis (brain hemorrhage)
Paralysis (brain hemorrhage)
Paralysis (brain hemorrhage)
AIDS
AIDS
AIDS
AIDS
AIDS
AIDS
AIDS
AIDS
AIDS
AIDS
Diabetes
Diabetes
Diabetes
Diabetes
D

In [176]:
import pickle

In [177]:
with open('model.pkl','wb') as file:
    pickle.dump(model,file)

In [178]:
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# TESTING THE MODEL

In [179]:
# Function to get the key (disease name) for a given value (predicted value)
def get_disease_by_value(disease_map, value):
    for key, val in disease_map.items():
        if val == value:
            return key
    return "Disease not found"

# Example disease map (as provided)

diseasemap = {
    '(vertigo) Paroymsal  Positional Vertigo': 0, 'AIDS': 1, 'Acne': 2, 
    'Alcoholic hepatitis': 3, 'Allergy': 4, 'Arthritis': 5, 'Bronchial Asthma': 6,
    'Cervical spondylosis': 7, 'Chicken pox': 8, 'Chronic cholestasis': 9, 
    'Common Cold': 10, 'Dengue': 11, 'Diabetes': 12, 'Dimorphic hemmorhoids(piles)': 13, 
    'Drug Reaction': 14, 'Fungal infection': 15, 'GERD': 16, 'Gastroenteritis': 17, 
    'Heart attack': 18, 'Hepatitis B': 19, 'Hepatitis C': 20, 'Hepatitis D': 21, 
    'Hepatitis E': 22, 'Hypertension': 23, 'Hyperthyroidism': 24, 'Hypoglycemia': 25, 
    'Hypothyroidism': 26, 'Impetigo': 28, 'Jaundice': 29, 'Malaria': 30, 'Migraine': 31, 
    'Osteoarthristis': 32, 'Paralysis (brain hemorrhage)': 33, 'Peptic ulcer disease': 34, 
    'Pneumonia': 35, 'Psoriasis': 36, 'Tuberculosis': 37, 'Typhoid': 38, 
    'Urinary tract infection': 39, 'Varicose veins': 40, 'hepatitis A': 41
}

# Assuming model.predict() returns an array of predicted values

predicted_values = loaded_model.predict([[0 ,0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,0, 0, 0, 0, 0,0,0,0,0,0,0]])

# Iterate through predicted values and get corresponding disease names
predicted_diseases = [get_disease_by_value(diseasemap, value) for value in predicted_values]
#print(predicted_diseases)

# Checking and printing corresponding medications from the 'medicine' DataFrame
for disease in predicted_diseases:
    if disease in medicine['Disease'].values:
        # Get the corresponding medication
        medication = medicine.loc[medicine['Disease'] == disease, 'Medication'].values[0]
        print(f"Disease: {disease}, Medication: {medication}")
        
    if disease in precaution['Disease'].values:
        precautions= precaution.loc[precaution['Disease']== disease, ['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']].values[0]
        print(f"Precaution:{precautions[0]},{precautions[1]},{precautions[2]},{precautions[3]}")
        
    if disease in diet['Disease'].values:
        diets=diet.loc[diet['Disease']==disease,'Diet'].values[0]
        print(f"Diet:{diets}")
        
    if disease in grouped_workout['disease'].values:
        wo = grouped_workout.loc[grouped_workout['disease'] == disease, 'workout'].values[0]
        print(f"workout:{wo}\n")
    else:
        print(f"Disease: {disease}, Medication not found")


Disease: Typhoid, Medication: ['Vaccination', 'Antiviral drugs', 'IV fluids', 'Blood transfusions', 'Liver transplant']
Precaution:eat high calorie vegitables,antiboitic therapy,consult doctor,medication
Diet:['Typhoid Diet', 'High-Calorie Diet', 'Soft and bland foods', 'Hydration', 'Protein-rich foods']
workout:['Consume easily digestible foods', 'Stay hydrated', 'Include nutrient-rich foods', 'Consume foods rich in vitamin C and antioxidants', 'Include protein-rich foods', 'Avoid fatty and greasy foods', 'Consult a healthcare professional', 'Gradually resume normal diet', 'Follow medical recommendations', 'Avoid alcohol and caffeine']



c:\Users\lokes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [180]:
#(encoded_df==0).all(axis = 1).any()

In [181]:
#predicted_values

In [182]:
!pip install streamlit

In [183]:
!pip install streamlit --quiet

In [184]:
pip show scikit-learn


Name: scikit-learn
Version: 1.5.2
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: BSD 3-Clause License

Copyright (c) 2007-2024 The scikit-learn developers.
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation
  and/or other materials provided with the distribution.

* Neither the name of the copyright holder nor the names of its
  contributors may be used to endorse or promote products derived from
  this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS 

In [185]:
pip install scikit-learn==<version_number>


Note: you may need to restart the kernel to use updated packages.


The syntax of the command is incorrect.


In [186]:
import joblib
# Assuming model is your trained model
joblib.dump(model, 'model.pkl')


['model.pkl']

In [187]:
import pickle
# Saving model
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Loading model
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)


In [188]:
pip install openpyxl

In [189]:
from openpyxl import load_workbook

# Load the workbook and select the sheet
workbook = load_workbook('output.xlsx')
sheet = workbook['Sheet1']  # Replace with your sheet name

# Get column headers
column_names = [cell.value for cell in sheet[1]]  # Assuming the first row has headers
print("Column names:", column_names)


Column names: [' acidity', ' back_pain', ' bladder_discomfort', ' breathlessness', ' burning_micturition', ' chest_pain', ' chills', ' constipation', ' continuous_sneezing', ' cough', ' cramps', ' fatigue', ' headache', ' high_fever', ' indigestion', ' joint_pain', ' mood_swings', ' muscle_wasting', ' muscle_weakness', ' neck_pain', ' pain_during_bowel_movements', ' patches_in_throat', ' pus_filled_pimples', ' shivering', ' skin_rash', ' stiff_neck', ' stomach_pain', ' sunken_eyes', ' vomiting', ' weakness_in_limbs', ' weight_gain', ' weight_loss', ' yellowish_skin', 'itching', ' abdominal_pain', ' acidity', ' anxiety', ' blackheads', ' bladder_discomfort', ' blister', ' breathlessness', ' bruising', ' chest_pain', ' chills', ' cold_hands_and_feets', ' cough', ' cramps', ' dehydration', ' dizziness', ' fatigue', ' foul_smell_of urine', ' headache', ' high_fever', ' indigestion', ' joint_pain', ' knee_pain', ' lethargy', ' loss_of_appetite', ' mood_swings', ' nausea', ' neck_pain', ' no